In [1]:
# A few imports
import yt
import numpy as np
import trident
import matplotlib.pyplot as pl
from yt.units import kpc,mp
from yt.fields.particle_fields import add_volume_weighted_smoothed_field

In [2]:
%%time
# Import some FIRE2 data
fname = '/mnt/raid-project/murray/lakhlani/FIRE2_core/m12c_res56000/output/snapshot_600.hdf5'
ds = yt.load(fname)
# The center of the Halo was recovered in another notebook
center_Halo = [25277.66673046,34505.21241664,32868.48520185]
sp = ds.sphere(center_Halo, (10, "kpc"))
ang_mom_sphere = ds.sphere(center_Halo, (0.5, "kpc"))

yt : [INFO     ] 2020-05-20 11:52:36,046 Calculating time from 1.000e+00 to be 4.355e+17 seconds
yt : [INFO     ] 2020-05-20 11:52:36,047 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2020-05-20 11:52:36,085 Parameters: current_time              = 4.3545571088051386e+17 s
yt : [INFO     ] 2020-05-20 11:52:36,086 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2020-05-20 11:52:36,087 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-05-20 11:52:36,088 Parameters: domain_right_edge         = [60000. 60000. 60000.]
yt : [INFO     ] 2020-05-20 11:52:36,089 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2020-05-20 11:52:36,090 Parameters: current_redshift          = 0.0
yt : [INFO     ] 2020-05-20 11:52:36,091 Parameters: omega_lambda              = 0.728
yt : [INFO     ] 2020-05-20 11:52:36,092 Parameters: omega_matter              = 0.272
yt : [INFO     ] 2020-05-20 11:52:36,092 Parameters: omega_radiation           = 0.

CPU times: user 7.27 s, sys: 1.84 s, total: 9.11 s
Wall time: 9.11 s


In [3]:
def find_normal_vector():
    # Finds the galaxy's normal vector by averaging the angular momentas
    # of the particles
    angular_momentum =[np.mean(ang_mom_sphere['angular_momentum_x']),
                       np.mean(ang_mom_sphere['angular_momentum_y']),
                       np.mean(ang_mom_sphere['angular_momentum_z'])]
    normalized_ang = np.array(angular_momentum)/np.linalg.norm(angular_momentum)
    print(normalized_ang)

In [4]:
# Define a few vectors. parr is along the galaxy plane. 
# Perp is perpendicular to the galaxy plane
norm_perp = np.array([0.91752181,-0.21788365,-0.33268671])
norm_parr = np.array([-0.21788365,-0.91752181,0])/np.linalg.norm([-0.21788365,-0.91752181,0])

In [5]:
# Creates the field which will be required to compute the dispersion measure
def _ElectronDensity(field, data):
    xh= 1. - data['PartType0','Metallicity_00'] - data['PartType0','Metallicity_01']
    ne= xh * data['PartType0','Density'] * data['PartType0','ElectronAbundance'] / mp
    return ne
# Adds the field to the data set
ds.add_field(('PartType0', '_ElectronDensity'), function = _ElectronDensity, 
             sampling_type = 'particle', units='1/cm**3')

# smooths a particle field into a continuous field
fn = add_volume_weighted_smoothed_field(
      "PartType0", "particle_position", "particle_mass", "smoothing_length",
      "density", "_ElectronDensity",ds.field_info)

# Labels the new field
ds.field_info.alias(('gas', '_ElectronDensity'), fn[0])
ds.derived_field_list.append(('gas', '_ElectronDensity'))

In [14]:
def make_ray_plots(projection_axis, ray_axis, filename, all_angles = False):
    ''' 
    This function computes the dispersion measure along a light ray
    going through a simulated galaxy.
    Arguments:
    projection_axis: vector perpendicular to the light ray used to make a projection plot
    ray_axis: vector parallel to the light ray
    filename: name inditifer for images
    all_angles: Instead of creating a single off-axis projection plot,
    the function creates three on-axis projection plots when True
    '''
    
    # RAY CREATION
    ray_end = list(np.array(center_Halo) + 4*np.array(ray_axis))
    ray = trident.make_simple_ray(ds,
                                   start_position=center_Halo,
                                    end_position=ray_end,
                                    fields = ['_ElectronDensity','density','temperature'],
                                    data_filename="{}.h5".format(filename))
    
    # GALAXY PLOTS
    if all_angles:
        # Creates three on-axis projection plots
        axes=['x','y','z']
        for axis in axes:
            prj_x = yt.ProjectionPlot(ds, axis, 'density', width=10*kpc, data_source = sp,
                                      center=center_Halo)
            prj_x.annotate_ray(ray)
            prj_x.save(filename+'_galaxy_{}.pdf'.format(str(axis)))
        
    else:
        image = yt.OffAxisProjectionPlot(ds, projection_axis, fields= 'density', 
                                         center= center_Halo, width=10, data_source = sp)
        image.annotate_ray(ray)
        image.save(filename+'_galaxy.pdf')
    
        # COMPUTING THE DM
        length = np.linalg.norm(np.array(center_Halo)-np.array(ray_end))
        int_elec_dens = []
        int_total = 0
        increment = length/len(ray.data['_ElectronDensity'])
        # Finding the entry where arc_length>1kpc:
        for i in range(len(ray.data['_ElectronDensity'])):
            # Convert in pc/cm^-3
            int_total += ray.data['_ElectronDensity'][i]*increment*1000
            int_elec_dens.append(float(int_total))
        
        # LIGHT RAY DATA PLOTS
        fig, axs = pl.subplots(2,2)

        axs[0,0].plot(np.linspace(0,length,len(ray.data['density'])), 
                      np.log10(ray.data['density']))
        axs[0,0].set_xlabel('Arc Length (kpc)')
        axs[0,0].set_ylabel('Density '+r'$\log\left(\frac{g}{cm^3}\right)$')

        axs[0,1].plot(np.linspace(0,length,len(ray.data['temperature'])), 
                      np.log10(ray.data['temperature']))
        axs[0,1].set_xlabel('Arc Length (kpc)')
        axs[0,1].set_ylabel('Temperature '+r'$\log(K)$')

        axs[1,0].plot(np.linspace(0,length,len(ray.data['_ElectronDensity'])), 
                np.log10(ray.data['_ElectronDensity']))
        axs[1,0].set_xlabel('Arc Length (kpc)')
        axs[1,0].set_ylabel('Electron Density '+r'$\log(cm^{-3})$')

        axs[1,1].plot(np.linspace(0,length,len(int_elec_dens)), np.log10(int_elec_dens))
        axs[1,1].set_xlabel('Arc Length (kpc)')
        axs[1,1].set_ylabel('DM '+r'$\log(cm^{-2})$')

        pl.subplots_adjust(left = 0.20,right = 0.9,bottom = 0.2,top = 0.9,
                           wspace = 0.4,hspace = 0.8)
        fig.savefig(filename+'_subplots.pdf')
    return ray

In [13]:
%%time
ray_parr = make_ray_plots(norm_perp, norm_parr, 'ray_parr', all_angles=True)

yt : [INFO     ] 2020-05-20 12:14:04,278 Getting segment at z = 0.0: [0.42129445 0.57508687 0.54780809] unitary to [0.42127904 0.57502201 0.54780809] unitary.
/home/hlamarre/.local/lib/python3.7/site-packages/yt/units/yt_array.py:1417: RuntimeWarning: invalid value encountered in true_divide
  out=out, **kwargs)
/home/hlamarre/.local/lib/python3.7/site-packages/yt/units/yt_array.py:1417: RuntimeWarning: invalid value encountered in less
  out=out, **kwargs)
yt : [INFO     ] 2020-05-20 12:14:04,285 Getting subsegment: [0.4212944455076667 unitary, 0.5750868736106668 unitary, 0.5478080866975 unitary] to [0.42127904253795223 unitary, 0.5750220107324665 unitary, 0.5478080866975 unitary].
yt : [INFO     ] 2020-05-20 12:16:35,729 Saving field data to yt dataset: ray_parr.h5.
yt : [INFO     ] 2020-05-20 12:16:35,931 Parameters: current_time              = 4.3545571088051386e+17 s
yt : [INFO     ] 2020-05-20 12:16:35,932 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2020-05-2

CPU times: user 2min 57s, sys: 1min 26s, total: 4min 24s
Wall time: 4min 23s


In [15]:
%%time
ray_perp = make_ray_plots(norm_parr, norm_perp, 'ray_perp')

yt : [INFO     ] 2020-05-20 12:21:23,609 Getting segment at z = 0.0: [0.42129445 0.57508687 0.54780809] unitary to [0.42135561 0.57507235 0.54778591] unitary.
yt : [INFO     ] 2020-05-20 12:21:23,612 Getting subsegment: [0.4212944455076667 unitary, 0.5750868736106668 unitary, 0.5478080866975 unitary] to [0.42135561362833335 unitary, 0.575072348034 unitary, 0.5477859075835 unitary].
yt : [INFO     ] 2020-05-20 12:24:00,754 Saving field data to yt dataset: ray_perp.h5.
yt : [INFO     ] 2020-05-20 12:24:00,959 Parameters: current_time              = 4.3545571088051386e+17 s
yt : [INFO     ] 2020-05-20 12:24:00,960 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2020-05-20 12:24:00,961 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2020-05-20 12:24:00,962 Parameters: domain_right_edge         = [60000. 60000. 60000.] code_length
yt : [INFO     ] 2020-05-20 12:24:00,963 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2020-05-20 12

AttributeError: 'ImageArray' object has no attribute 'annotate_ray'